In [6]:
import numpy as np
import pandas as pd
from math import sqrt
from pprint import pprint
from sklearn import datasets
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [7]:
# Carga de datos.
datos = pd.read_csv('data/boston.csv', sep="\s+", skiprows=22, header=None)
X = np.hstack([datos.values[::2, :], datos.values[1::2, :2]])
y = datos.values[1::2, 2]
# pprint(datos)
# print(np.shape(datos.data))

In [8]:
# Partición EXTERNA. Test: hold-out split 80-20%.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Train dimensions: ', np.shape(X_train))
print('Test dimensions:  ', np.shape(X_test))

Train dimensions:  (193, 16)
Test dimensions:   (49, 16)


In [9]:
# Algoritmo de aprendizaje.
reg = DummyRegressor()

In [10]:
# Métricas de evaluación.
metricas = {
  'MAE': 'neg_mean_absolute_error',
  'RMSE': make_scorer(
          lambda y, y_pred:
            sqrt(mean_squared_error(y, y_pred)),
            greater_is_better=False),
  'MAPE': make_scorer(
          lambda y, y_pred:
            np.mean(np.abs((y - y_pred) / y)) * 100,
            greater_is_better=False)}

In [11]:
# Validación y evaluación del modelo.
# en "cv = KFold(n_splits=5)" se hace un cross-validation INTERNO!! 
evaluacion = cross_validate(reg, X_train, y_train, cv = KFold(n_splits=5), scoring = metricas)

In [12]:
# Presentación de los resultados de la evaluación.
pprint(evaluacion)

{'fit_time': array([0.00049686, 0.00024033, 0.00038981, 0.0001688 , 0.000108  ]),
 'score_time': array([0.00053811, 0.00020194, 0.00032806, 0.00040817, 0.00016689]),
 'test_MAE': array([-6.64609391, -5.78759241, -5.42780386, -7.50750255, -6.8984618 ]),
 'test_MAPE': array([-158.66288532, -106.80499531,  -81.96206125, -143.75384309,
       -134.55602044]),
 'test_RMSE': array([-6.95567472, -6.71661959, -6.01450409, -7.94278229, -7.47024411])}


In [13]:
# Veamos como disminuye el error durante el cross-validation interno si utilizamos un algoritmo más complejo
# El objetivo es utilizar cross-validation INTERNO para optimizar los hiperpárametros (C, por ejemplo)
from sklearn.svm import SVR
svr = SVR(C=100)
results = cross_validate(svr, X_train, y_train, cv=KFold(n_splits=5), scoring=metricas)
pprint(results)

{'fit_time': array([0.00134611, 0.00107217, 0.0010078 , 0.00102091, 0.00110698]),
 'score_time': array([0.00038385, 0.00034595, 0.00032115, 0.00032902, 0.00043178]),
 'test_MAE': array([-1.7410281 , -2.57794065, -1.80956293, -2.17392931, -1.91162999]),
 'test_MAPE': array([-30.56545136, -37.18484765, -21.26128543, -24.98108023,
       -21.76733774]),
 'test_RMSE': array([-2.90122335, -4.08992373, -3.00215786, -4.44496437, -3.26191229])}


In [14]:
# Una vez hemos optimizado los hiperparámetros en el conjunto de validación, entrenamos un nuevo modelo que contenga
# todos los datos de entrenamiento (es decir, train + validation) para evaluar el test
from sklearn.metrics import mean_absolute_error
svr = SVR(C=100) # Definimos el modelo con los hiperparámetros optimizados
svr = svr.fit(X_train, y_train) # Entrenamos el modelo

# Extraemos las predicciones del test
y_pred = svr.predict(X_test)
# Observamos el error cometido (En este caso utilizando solo la métrica MAE)
print('Error MAE en test: ', mean_absolute_error(y_pred, y_test))

Error MAE en test:  1.5136374850668557


In [15]:
# También podemos ver cuánto difieren los valores predichos de los reales (observamos únicamente los 10 primeros valores)
print('Predicciones:   ', np.round(y_pred[:10], 1))
print('Valores reales: ', y_test[:10])

Predicciones:    [ 8.2 11.2  6.6 18.1 11.1 18.3 18.2  6.8  5.3  6.4]
Valores reales:  [10.81  5.96  7.38 18.1   9.69 18.1  18.1   7.38  6.91  5.86]
